In [ ]:
!unzip '/content/drive/MyDrive/Nature_Data/Train.zip'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,BatchNormalization,Dropout

In [ ]:

train_path='/content/seg_train'
test_path='/content/seg_test'

In [ ]:
train_datagen =ImageDataGenerator(rescale=1./255, validation_split=0.25)


In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_path,  # Source directory
        target_size=(150, 150),  # Resizes images
        batch_size=15,
        class_mode='categorical',subset = 'training')

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=15,
    class_mode='categorical',
    subset='validation') # set as validation data

In [ ]:
def Classifier():
    model=Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.25))
    model.add(Conv2D(32,(3,3),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.25))
    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.25))

    model.add(Conv2D(128,(3,3),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(720,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(6,activation='softmax'))
    model.compile(loss='categorical_crossentropy',
    optimizer='Adam',metrics=['accuracy'])

    return model

Classifier=Classifier()

In [ ]:
Classifier.fit(train_generator,
          validation_data=validation_generator,
          steps_per_epoch=train_generator.samples/train_generator.batch_size,
          validation_steps=validation_generator.samples/validation_generator.batch_size,
          epochs=25)

In [ ]:
Classifier.save("model_1.h5")

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/model_1.h5')

In [ ]:
old_classes=train_generator.class_indices
classes= dict([(value, key) for key, value in old_classes.items()])
print(classes)

In [ ]:
import cv2
import tensorflow as tf
import numpy as np


# img=cv2.imread('/content/seg_test/forest/20056.jpg')
test_image = tf.keras.preprocessing.image.load_img('/content/seg_pred/10712.jpg',target_size = (150, 150)) # IT IS LOADING STORED IMAGE FROM CAM_IMAGES FOLDER
test_image = tf.keras.preprocessing.image.img_to_array(test_image)  # IT IS CONVERTING IMAGE TO ARRAY FROMATE
test_image = np.expand_dims(test_image, axis = 0)  # IT IS EXPANDING DIMENTIONS OF IMAGE LIKE (1,224,224,3)

Predictions=model.predict(test_image)
Max_val=np.argmax(Predictions) # IT GIVES THE MAX_VALUE FROM PREDICTIONS
print('Predicted_Item :',classes[Max_val])

In [ ]:
from google.colab.patches import cv2_imshow
# /content/seg_pred/10079.jpg, /content/seg_pred/10092.jpg, /content/seg_pred/10164.jpg,/content/seg_pred/10013.jpg,/content/seg_pred/1003.jpg, /content/seg_pred/10712.jpg
img=cv2.imread('/content/seg_pred/10712.jpg')
cv2_imshow(img)